In [ ]:
!pip install scapy
from scapy.all import *

## Bienvenidxs al taller de Wiretapping 2023.

A lo largo del taller van a poder probar los diferentes comandos que nos provee la librería Scapy. El objetivo es que al finalizar el taller tengan el código necesario para poder hacer los experimentos y el informe.

# Jupyter Notebook

Para empezar, vamos a utilizar un notebook de Jupyter, una herramienta para correr Python **muy** recomendada, donde pueden escribir y correr código Python. Cada sección donde hay código se le llama **celda** (Cell, en inglés).
- Para ejecutar el código que hay dentro de una celda se utiliza `Cntrl + Enter`.

# Scapy
* Es una librería de Python.
* Es muy útil para:
    * Crear paquetes de red.
    * Decodificar paquetes.
    * Capturar.
    * Enviar y recibir paquetes (próximo taller, stay tuned).
    
Documentación (**recomendada**): https://scapy.readthedocs.io/en/latest/    
Para saber más:    

## Ej. 1: Utilizando Scapy ver cómo es el formato del paquete Ether y ARP
1. `ls()` para ver todos los tipos de paquetes que reconoce Scapy.
2. `ls(PROTOCOLO)` si reemplazamos PROTOCOLO por algún protocolo (ej: IP, ETHER) podremos ver el formato del paquete. Probar ls(IP) o ls(Ether)

## Ej. 2: Crear un paquete, codificarlo y decodificarlo.

En el siguiente ejercicio tendremos que asignar a una variable un paquete de Ethernet (consultar en la documentación), convertirlo a bytes (recuerden que los símbolos que recibe la interfaz de red no son más que bits que luego son interpretados) para luego reconvertirlo a un paquete legible.

*Hint:
Para ver en formato bytes utilizamos `raw(FRAME)`, para ver en un formato más amigable `hexdump(FRAME)`. Luego para reconstruirlo podremos hacer `Ether(formato_en_bytes)`.*

In [ ]:
IP(raw(IP()))

## Ej. 3: Inspeccionando frames

### Ej. 3.1: Dado un paquete Ethernet , imprimir con la función print(), su destino (dst), fuente (src) y número de protocolo superior (type)

In [ ]:
# Los protocolos separados por "/" significa que estoy encapsulando uno dentro del otro

frame = Ether()/IP(dst="www.dc.uba.ar")/TCP()/"GET /index.html HTTP/1.0 \n\n"
print(frame[IP]) # Observar que la manera de acceder a un protocolo encapsulado es de la manera frame[PROTOCOLO].

### Ej 3.2: Dado una lista de paquetes, filtrar por protocol.

PCAP (Package CAPture) es un formato muy utilizado para capturar tráfico de la red. Al utilizar Sniffers como Wireshark, podremos exportar el dump en formato `pcap` y consumirlo desde Scapy. Lo que devuelve `rdpcap` es una lista de frames la cual puede iterarse como cualquier lista.

Como vimos en el ejercicio 3.1 uno puede dado un paquete, pedir su destino, fuente e incluso el tipo (*layer*) de protocolo.

* Los paquetes de Scapy proveen la función `haslayer(layer)`. Esto devuelve un booleano si dentro del paquete se encuentra el `layer`. Si a un paquete `pkt = Ether()` le pedimos `pkt.haslayer(IP)` devuelve falso. Incluso, los paquetes permiten indexar por protocolo.


* Podremos pedirle al paquete **pkt** `pkt[IP]` y devolverá únicamente el paquete IP y lo que este encapsula. Tener en cuenta que `pkt[IP].dst` nos devolverá un resultado distinto que `pkt[Ether].dst`.


Para este ejercicio:

- Contar cuantos paquetes contienen a la capa Ethernet.
- Contar cuantas direcciones destino distintas de capa 2 hay.
- Enumerar los distintos tipos de protocolos que encapsula la capa 2.

In [ ]:
#dump_de_frames =rdpcap("./dump_prueba.pcap")
#
# for p in dump_de_frames:
#     print(p[Ether].type)
#
# print(frame[Ether].dst)
# frame[IP].dst

## Ej. 4: Sniffing

Scapy es una herramienta que además de permitir inspeccionar paquetes, permite hacer lo mismo que Wireshark desde el mismo código. De esta manera podremos capturar paquetes e inspeccionarlos sin salir de nuestro programa de Python.

Veamos que hace la herramienta `sniff`.

In [ ]:
help(sniff)

### Ej 4.1: Contar Broadcast & Unicast

Utilizando la función `sniff(count=10)`, por cada paquete nuevo, que se invoque a una segunda función implementada por nosotrxs y que cuente para aquellos paquetes de la capa Ethernet, cuántos son de Broadcast (`dst=ff:ff:ff:ff:ff:ff`) y cuántos Unicast.

In [ ]:
def funcion():
#     Renombrar función
#     print(algo)

### Ej 4.2 Contar protocolos de la capa superior

Además utilizando el campo `type` del paquete de Ethernet, mostrar qué protocolos de la capa superior inmediata aparecen y cuántos por cada uno.

# Trabajo práctico

Sean p1..pn las tramas de capa 2 que se capturan en una red local. Se pueden modelar las tramas capturadas como una fuente de información de memoria nula S<sub>1</sub> = {s<sub>1</sub>, s<sub>2</sub>, ..., s<sub>q</sub>}, donde cada si está formado por la combinación entre el tipo de destino de la trama (Unicast o Broadcast) y el protocolo de la capa inmediata superior encapsulado en la misma. Por ejemplo, s<sub>i</sub> = < Broadcast, ARP >.

A continuación se presenta un código Python de ejemplo para capturar de paquetes y calcula las probabilidades de cada uno de los símbolos de la fuente de información S1 en una red con la que tenemos conexión.

In [ ]:
S1 = {}
def mostrar_fuente(S):
    N = sum(S.values())
    simbolos = sorted(S.items(), key=lambda x: -x[1])
    print("\n".join([ " %s : %.5f" % (d,k/N) for d,k in simbolos ]))
    print()
def callback(pkt):
    if pkt.haslayer(Ether):
        dire = "BROADCAST" if pkt[Ether].dst=="ff:ff:ff:ff:ff:ff" else "UNICAST"
        proto = pkt[Ether].type # El campo type del frame tiene el protocolo
        s_i = (dire, proto) # Aca se define el simbolo de la fuente
        if s_i not in S1:
            S1[s_i] = 0.0
        S1[s_i] += 1.0
    mostrar_fuente(S1)

sniff(prn=callback)

En dicha salida se muestran los símbolos de la fuente de información S1 y sus respectivas probabilidades.
Cada símbolo es una tupla que indica si se trata de paquetes **Broadcast** o **Unicast** y el **protocolo de capa
superior** al que corresponde cada paquete capturado. Por ejemplo, 2048 es IP, de los que se capturaron sólo
paquete Broadcast, 2054 es ARP para los cuales hay tanto Broadcast como Unicast, etc.

Probar la captura de paquetes usando el código presentado anteriormente, que captura tráfico en una red local y muestra representativamente la fuente modelada S1.

La salida consiste en una tabla que muestra la probabilidad de cada símbolo de la fuente. Luego, extender el código para que calcule la información de cada símbolo y la entropía de la fuente. Finalmente, realizar una captura de tráfico utilizando el código extendido anteriormente. La captura deben ser lo más extensa posibles (por ejemplo de más de 10.000 tramas).

### Info util para Python

Si utilizamos un editor de texto, luego para correr el archivo corremos en la terminal
```
$python3 archivo.py
```

Si queremos declarar funciones y luego usarlas en un main, debemos declarar la función main de la siguiente manera:
    
```
def main() -> int:
    phrase = "hola"
    echo(phrase)
    return 0

if __name__ == '__main__':
    main()
```

También se puede usar Jupyter Notebook.

In [ ]:
sniff()

In [ ]:
lsc()